In [1]:
import geopandas
import folium
import json
import numpy
import pulp
import routingpy
from routingpy import OSRM
import shapely
from shapely.geometry import shape, Point
import spopt
from spopt.locate import MCLP, simulated_geo_points
import pandas as pd
import pprint
import json

In [2]:
df = geopandas.read_file('강서구 행정경계 포인트.shp')#.shx file도 같이 필요
SF_MISSION_BERNAL = {
    "type": "Polygon",
    "coordinates": [
        pd.concat([df['x'], df['y']], axis=1).values.tolist()
    ],
}
SF_MISSION_BERNAL = str(SF_MISSION_BERNAL).replace("'", '"')

In [3]:
df = shape(json.loads(SF_MISSION_BERNAL))
df = geopandas.GeoDataFrame(geometry=[df])
df

,geometry
0,"POLYGON ((126.80902 37.60064, 126.80936 37.600..."


In [4]:
df_b = pd.read_csv('의료시설_포인트.csv', encoding='euc-kr')
df_b = pd.concat([df_b['x'], df_b['y']], axis=1).values.tolist()
NUM_BOXJI = len(df_b)

df_b2 = pd.read_csv('노인복지시설_포인트.csv', encoding='euc-kr')
df_b2 = pd.concat([df_b2['x'], df_b2['y']], axis=1).values.tolist()
NUM_NOIN = len(df_b2)

df_b3 = pd.read_csv('버스정류소_포인트.csv', encoding='utf8')
df_b3 = pd.concat([df_b3['x'], df_b3['y']], axis=1).values.tolist()
NUM_BUS = len(df_b3)

df_b4 = pd.read_csv('공공시설_포인트.csv', encoding='utf8')
df_b4 = pd.concat([df_b4['x'], df_b4['y']], axis=1).values.tolist()
NUM_GONG = len(df_b4)

# 영유아
# 문화및집회시설
# 의원
# 주차장
# 판매및영업시설
# 학군

NUM_SISER = NUM_BOXJI + NUM_NOIN + NUM_BUS + NUM_GONG

In [5]:
m = folium.Map(location=[37.551111918342,126.84930138784], zoom_start=13)

folium.GeoJson(
    data=SF_MISSION_BERNAL,
    style_function=lambda x: {"color": "darksalmon", "fillColor": "darksalmon"},
).add_to(m)

display(m)

In [6]:
boxji = folium.FeatureGroup("boxji").add_to(m)

for coord in df_b:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=3,
        fill=True,
        fill_opacity=1,
        color="cornflowerblue",
    ).add_to(boxji)

for coord in df_b2:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=3,
        fill=True,
        fill_opacity=1,
        color="white",
    ).add_to(boxji)
    
for coord in df_b3:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=3,
        fill=True,
        fill_opacity=1,
        color="gray",
    ).add_to(boxji)
    
for coord in df_b4:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=3,
        fill=True,
        fill_opacity=1,
        color="pink",
    ).add_to(boxji)

In [7]:
noriter = pd.read_csv('공원_포인트.csv', encoding='utf8')
noriter = pd.concat([noriter['x'], noriter['y']], axis=1).values.tolist()

In [8]:
noriters = folium.FeatureGroup("noriters").add_to(m)

for coord in noriter:
    folium.CircleMarker(
        location=[coord[1], coord[0]],
        radius=5,
        fill=True,
        fill_opacity=1,
        color="green",
    ).add_to(noriters)

In [9]:
display(m)

In [10]:
all_locations = df_b + df_b2 + df_b3 + df_b4 + noriter
source_indices = [i for i in range(NUM_SISER)]
target_indices = [i for i in range(NUM_SISER, len(all_locations))]

In [11]:
#client = OSRM(base_url="https://router.project-osrm.org")
#osrm_routing_matrix = client.matrix(
#    locations=all_locations,
#    sources=source_indices,
#    destinations=target_indices,
#    profile="pedestrian",
#)
#cost_matrix = numpy.array(osrm_routing_matrix.durations)
test1 = pd.read_csv('공원_의료시설_거리행렬.csv', encoding='utf8')
test_df = pd.DataFrame()
for i in test1['origin_id'].unique():
    test_df[i] = test1[test1['origin_id'] == i]['total_cost'].values.tolist()
cost_matrix1 = numpy.array(test_df.values.tolist())

test2 = pd.read_csv('공원_노인복지시설_거리행렬.csv', encoding='utf8')
test_df2 = pd.DataFrame()
for i in test2['origin_id'].unique():
    test_df2[i] = test2[test2['origin_id'] == i]['total_cost'].values.tolist()
cost_matrix2 = numpy.array(test_df2.values.tolist())

test3 = pd.read_csv('공원_버스정류소_거리행렬.csv', encoding='utf8')
test_df3 = pd.DataFrame()
for i in test3['origin_id'].unique():
    test_df3[i] = test3[test3['origin_id'] == i]['total_cost'].values.tolist()
cost_matrix3 = numpy.array(test_df3.values.tolist())

test4 = pd.read_csv('공원_공공시설_거리행렬.csv', encoding='utf8')
test_df4 = pd.DataFrame()
for i in test4['origin_id'].unique():
    test_df4[i] = test4[test4['origin_id'] == i]['total_cost'].values.tolist()
cost_matrix4 = numpy.array(test_df4.values.tolist())


cost_matrix =  numpy.vstack([cost_matrix1, cost_matrix2, cost_matrix3, cost_matrix4])

C:\Users\User\AppData\Local\Temp\ipykernel_11648\1235079802.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[i] = test1[test1['origin_id'] == i]['total_cost'].values.tolist()
C:\Users\User\AppData\Local\Temp\ipykernel_11648\1235079802.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[i] = test1[test1['origin_id'] == i]['total_cost'].values.tolist()
C:\Users\User\AppData\Local\Temp\ipykernel_11648\1235079802.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `f

In [12]:
SERVICE_RADIUS = 1000
ai = numpy.hstack([numpy.full((1, NUM_BOXJI), 5, dtype=int), 
                   numpy.full((1, NUM_NOIN), 2, dtype=int), 
                   numpy.full((1, NUM_BUS), 4, dtype=int), 
                   numpy.full((1, NUM_GONG), 3, dtype=int)])
P_FACILITIES = 5

In [13]:
solver = pulp.COIN_CMD(msg=False)
mclp = MCLP.from_cost_matrix(cost_matrix, ai, SERVICE_RADIUS, p_facilities=P_FACILITIES)
mclp = mclp.solve(solver)

In [14]:
obj_val = int(mclp.problem.objective.value())
minutes = 0
obj_perc = mclp.perc_cov
print(
    f"Of the {NUM_SISER} siser, {obj_val} have access to at least 1 "
    f"noriter within a {minutes}-minute walk along the street network when "
    f"siting {P_FACILITIES} noriters. This accounts for {obj_perc}% coverage."
)

Of the 1027 siser, 1954 have access to at least 1 noriter within a 0-minute walk along the street network when siting 5 noriters. This accounts for 48.68549172346641% coverage.


In [15]:
sited_noriter_fg = folium.FeatureGroup("Sited noriter").add_to(m)
allocated_households_demand_fg = folium.FeatureGroup("Allocated siser").add_to(m)
solution = {}
colors = [
    '#F0785A',
    '#F07869',
    '#F07878',
    '#F07887',
    '#F07896',
    '#F078A5',
    '#F078B4',
    '#F078C3',
    '#F078D2',
    '#F078E1',
    '#F078F0',
    '#F078FF',
    '#F0871E',
    '#F0872D',
    '#F0873C',
    '#F0874B',
    '#F0875A',
    '#F08769',
    '#F08778',
    '#F08787',
    '#F08796',
    '#F087A5',
    '#F087B4',
    '#F087C3',
    '#F087D2',
    '#F087E1',
    '#F087F0',
    '#F087FF',
    '#F0961E',
    '#F0962D',
    '#F0963C',
    '#F0964B',
    '#F0965A',
    '#F09669',
    '#F09678',
    '#F09687',
    '#F09696',
    '#F096A5',
    '#F096B4',
    '#F096C3',
    '#F096D2',
    '#F096E1',
    '#F096F0',
    '#F096FF',
    '#F0A51E',
    '#F0A52D',
    '#F0A53C',
    '#F0A54B',
    '#F0A55A',
    '#F0A569',
    '#F0A578',
    '#F0A587',
    '#F0A596',
    '#F0A5A5',
    '#F0A5B4',
    '#F0A5C3',
    '#F0A5D2',
    '#F0A5E1',
    '#F0A5F0',
    '#F0A5FF',
    '#F0B41E',
    '#F0B42D',
    '#F0B43C',
    '#F0B44B',
    '#F0B45A',
    '#F0B469',
    '#F0B478',
    '#F0B487',
    '#F0B496',
    '#F0B4A5',
    '#F0B4B4',
    '#F0B4C3',
    '#F0B4D2',
    '#F0B4E1',
    '#F0B4F0',
    '#F0B4FF',
    '#F0C31E',
    '#F0C32D',
    '#F0C33C',
    '#F0C34B',
    '#F0C35A',
    '#F0C369',
    '#F0C378',
    '#F0C387',
    '#F0C396',
    '#F0C3A5',
    '#F0C3B4',
    '#F0C3C3',
    '#F0C3D2',
    '#F0C3E1',
    '#F0C3F0',
    '#F0C3FF',
    '#F0D21E',
    '#F0D22D',
    '#F0D23C',
    '#F0D24B',
    '#F0D25A',
    '#F0D269',
    '#F0D278',
    '#F0D287',
    '#F0D296',
    '#F0D2A5',
    '#F0D2B4',
    '#F0D2C3',
    '#F0D2D2',
    '#F0D2E1',
    '#F0D2F0',
    '#F0D2FF',
    '#F0E11E',
    '#F0E12D',
    '#F0E13C',
    '#F0E14B',
    '#F0E15A',
    '#F0E169',
    '#F0E178',
    '#F0E187',
    '#F0E196',
    '#F0E1A5',
    '#F0E1B4',
    '#F0E1C3',
    '#F0E1D2',
    '#F0E1E1',
    '#F0E1F0',
    '#F0E1FF',
    '#F0F01E',
    '#F0F02D',
    '#F0F03C',
    '#F0F04B',
    '#F0F05A',
    '#F0F069',
    '#F0F078',
    '#F0F087',
    '#F0F096',
    '#F0F0A5',
    '#F0F0B4',
    '#F0F0C3',
    '#F0F0D2',
    '#F0F0E1',
    '#F0F0F0',
    '#F0F0FF',
    '#F0FF1E',
    '#F0FF2D',
    '#F0FF3C',
    '#F0FF4B',
    '#F0FF5A',
    '#F0FF69',
    '#F0FF78',
    '#F0FF87',
    '#F0FF96',
    '#F0FFA5',
    '#F0FFB4',
    '#F0FFC3',
    '#F0FFD2',
    '#F0FFE1',
    '#F0FFF0',
    '#F0FFFF',
    '#FF1E1E',
    '#FF1E2D',
    '#FF1E3C',
    '#FF1E4B',
    '#FF1E5A',
    '#FF1E69',
    '#FF1E78',
    '#FF1E87',
    '#FF1E96',
    '#FF1EA5',
    '#FF1EB4',
    '#FF1EC3',
    '#FF1ED2',
    '#FF1EE1',
    '#FF1EF0',
    '#FF1EFF',
    '#FF2D1E',
    '#FF2D2D',
    '#FF2D3C',
    '#FF2D4B',
    '#FF2D5A',
    '#FF2D69',
    '#FF2D78',
    '#FF2D87',
    '#FF2D96',
    '#FF2DA5',
    '#FF2DB4',
    '#FF2DC3',
    '#FF2DD2',
    '#FF2DE1',
    '#FF2DF0',
    '#FF2DFF',
    '#FF3C1E',
    '#FF3C2D',
    '#FF3C3C',
    '#FF3C4B',
    '#FF3C5A',
    '#FF3C69',
    '#FF3C78',
    '#FF3C87',
    '#FF3C96',
    '#FF3CA5',
    '#FF3CB4',
    '#FF3CC3',
    '#FF3CD2',
    '#FF3CE1',
    '#FF3CF0',
    '#FF3CFF',
    '#FF4B1E',
    '#FF4B2D',
    '#FF4B3C',
    '#FF4B4B',
    '#FF4B5A',
    '#FF4B69',
    '#FF4B78',
    '#FF4B87',
    '#FF4B96',
    '#FF4BA5',
    '#FF4BB4',
    '#FF4BC3',
    '#FF4BD2',
    '#FF4BE1',
    '#FF4BF0',
    '#FF4BFF',
    '#FF5A1E',
    '#FF5A2D',
    '#FF5A3C',
    '#FF5A4B',
    '#FF5A5A',
    '#FF5A69',
    '#FF5A78',
    '#FF5A87',
    '#FF5A96',
    '#FF5AA5',
    '#FF5AB4',
    '#FF5AC3',
    '#FF5AD2',
    '#FF5AE1',
    '#FF5AF0',
    '#FF5AFF',
    '#FF691E',
    '#FF692D',
    '#FF693C',
    '#FF694B',
    '#FF695A',
    '#FF6969',
    '#FF6978',
    '#FF6987',
    '#FF6996',
    '#FF69A5',
    '#FF69B4',
    '#FF69C3',
    '#FF69D2',
    '#FF69E1',
    '#FF69F0',
    '#FF69FF',
    '#FF781E',
    '#FF782D',
    '#FF783C',
    '#FF784B',
    '#FF785A',
    '#FF7869',
    '#FF7878',
    '#FF7887',
    '#FF7896',
    '#FF78A5',
    '#FF78B4',
    '#FF78C3',
    '#FF78D2',
    '#FF78E1',
    '#FF78F0',
    '#FF78FF',
    '#FF871E',
    '#FF872D',
    '#FF873C',
    '#FF874B',
    '#FF875A',
    '#FF8769',
    '#FF8778',
    '#FF8787',
    '#FF8796',
    '#FF87A5',
    '#FF87B4',
    '#FF87C3',
    '#FF87D2',
    '#FF87E1',
    '#FF87F0',
    '#FF87FF',
    '#FF961E',
    '#FF962D',
    '#FF963C',
    '#FF964B',
    '#FF965A',
    '#FF9669',
    '#FF9678',
    '#FF9687',
    '#FF9696',
    '#FF96A5',
    '#FF96B4',
    '#FF96C3',
    '#FF96D2',
    '#FF96E1',
    '#FF96F0',
    '#FF96FF',
    '#FFA51E',
    '#FFA52D',
    '#FFA53C',
    '#FFA54B',
    '#FFA55A',
    '#FFA569',
    '#FFA578',
    '#FFA587',
    '#FFA596',
    '#FFA5A5',
    '#FFA5B4',
    '#FFA5C3',
    '#FFA5D2',
    '#FFA5E1',
    '#FFA5F0',
    '#FFA5FF',
    '#FFB41E',
    '#FFB42D',
    '#FFB43C',
    '#FFB44B',
    '#FFB45A',
    '#FFB469',
    '#FFB478',
    '#FFB487',
    '#FFB496',
    '#FFB4A5',
    '#FFB4B4',
    '#FFB4C3',
    '#FFB4D2',
    '#FFB4E1',
    '#FFB4F0',
    '#FFB4FF',
    '#FFC31E',
    '#FFC32D',
    '#FFC33C',
    '#FFC34B',
    '#FFC35A',
    '#FFC369',
    '#FFC378',
    '#FFC387',
    '#FFC396',
    '#FFC3A5',
    '#FFC3B4',
    '#FFC3C3',
    '#FFC3D2',
    '#FFC3E1',
    '#FFC3F0',
    '#FFC3FF',
    '#FFD21E',
    '#FFD22D',
    '#FFD23C',
    '#FFD24B',
    '#FFD25A',
    '#FFD269',
    '#FFD278',
    '#FFD287',
    '#FFD296',
    '#FFD2A5',
    '#FFD2B4',
    '#FFD2C3',
    '#FFD2D2',
    '#FFD2E1',
    '#FFD2F0',
    '#FFD2FF',
    '#FFE11E',
    '#FFE12D',
    '#FFE13C',
    '#FFE14B',
    '#FFE15A',
    '#FFE169',
    '#FFE178',
    '#FFE187',
    '#FFE196',
    '#FFE1A5',
    '#FFE1B4',
    '#FFE1C3',
    '#FFE1D2',
    '#FFE1E1',
    '#FFE1F0',
    '#FFE1FF',
    '#FFF01E',
    '#FFF02D',
    '#FFF03C',
    '#FFF04B',
    '#FFF05A',
    '#FFF069',
    '#FFF078',
    '#FFF087',
    '#FFF096',
    '#FFF0A5',
    '#FFF0B4',
    '#FFF0C3',
    '#FFF0D2',
    '#FFF0E1',
    '#FFF0F0',
    '#FFF0FF',
    '#FFFF1E',
    '#FFFF2D',
    '#FFFF3C',
    '#FFFF4B',
    '#FFFF5A',
    '#FFFF69',
    '#FFFF78',
    '#FFFF87',
    '#FFFF96',
    '#FFFFA5',
    '#FFFFB4',
    '#FFFFC3',
    '#FFFFD2',
    '#FFFFE1',
    '#FFFFF0',
    '#FFFFFF'
]
for i in range(len(noriter)):
    if mclp.fac2cli[i]:
        folium.CircleMarker(
            location=[noriter[i][1], noriter[i][0]],
            radius=25,
            fill=True,
            popup=folium.Popup(f"Sited noriter ({i})", show=True),
            color=colors[i],
        ).add_to(sited_noriter_fg)
        solution[i] = []
        for j in mclp.fac2cli[i]:
            solution[i].append((df_b + df_b2 + df_b3 + df_b4)[j])
            folium.CircleMarker(
                location=[(df_b + df_b2 + df_b3 + df_b4)[j][1], (df_b + df_b2 + df_b3 + df_b4)[j][0]],
                radius=10,
                fill=True,
                popup="siser",
                color=colors[i],
            ).add_to(allocated_households_demand_fg)
folium.LayerControl().add_to(m)

In [16]:
display(m)